In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 데이터 경로 설정
base_path = 'resized_skindata'  # 경로를 실제 Google Drive에 있는 skindata 폴더 경로로 변경하세요.

# 얼굴 부위 및 피부 상태 설정
face_parts = ['forehead', 'nose', 'cheek']
skin_states = ['dry', 'oily']

# 데이터 불러오기
data = []
labels = []

for part in face_parts:
    for state in skin_states:
        folder_path = os.path.join(base_path, 'train', state, part)
        for filename in os.listdir(folder_path):
            img = cv2.imread(os.path.join(folder_path, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR 형식이므로 RGB로 변환
            img = cv2.resize(img, (128, 128))  # 이미지 크기 조절

            data.append(img)
            labels.append(f'{state}_{part}')

# 데이터와 레이블을 numpy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 레이블 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)

# 데이터 분할 (8:2 비율)
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)


In [2]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
8/8 [==============================] - 33s 4s/step - loss: 198.6796 - accuracy: 0.1946 - val_loss: 4.8788 - val_accuracy: 0.2250
Epoch 2/10
8/8 [==============================] - 40s 5s/step - loss: 2.0690 - accuracy: 0.3389 - val_loss: 1.6370 - val_accuracy: 0.3083
Epoch 3/10
8/8 [==============================] - 30s 4s/step - loss: 1.4496 - accuracy: 0.4331 - val_loss: 3.5177 - val_accuracy: 0.2417
Epoch 4/10
8/8 [==============================] - 20s 2s/step - loss: 1.6290 - accuracy: 0.4728 - val_loss: 1.6238 - val_accuracy: 0.4000
Epoch 5/10
8/8 [==============================] - 19s 2s/step - loss: 1.1260 - accuracy: 0.5795 - val_loss: 1.6207 - val_accuracy: 0.3917
Epoch 6/10
8/8 [==============================] - 19s 2s/step - loss: 0.9985 - accuracy: 0.6172 - val_loss: 2.1274 - val_accuracy: 0.4250
Epoch 7/10
8/8 [==============================] - 20s 3s/step - loss: 0.9650 - accuracy: 0.6444 - val_loss: 3.6286 - val_accuracy: 0.2833
Epoch 8/10
3/8 [==========>.....

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# 테스트 데이터에 대한 예측
predictions = model.predict(X_test)

# 예측 결과 출력
for i in range(5):  # 예측 결과 중에서 처음 5개만 출력
    predicted_class = label_encoder.classes_[np.argmax(predictions[i])]
    actual_class = label_encoder.classes_[np.argmax(y_test[i])]
    print(f'Actual: {actual_class}, Predicted: {predicted_class}')
